In [2]:
import torch
import torch.nn as nn
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import gc
gc.collect()
torch.cuda.empty_cache()
cuda = torch.device('cuda') 
print(torch.__version__)
print(torchvision.__version__)
print(torch.__version__)

1.10.0+cu113
0.11.1+cu113
1.10.0+cu113


In [2]:
originalMobilenet = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

Using cache found in /home/plupiman/.cache/torch/hub/pytorch_vision_v0.10.0


In [3]:
#extracts the convolutional layers, ignoring the dense layer
mobilenetLayers = [module for module in originalMobilenet.children()]
initialLayers = nn.Sequential(*mobilenetLayers[0:-1])

[Sequential(
  (0): ConvNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): R

In [4]:
#Our mobilenet that has a new output space
class newMobilenet(nn.Module):
    def __init__(self, outputClasses, initialLayers):
        super(newMobilenet,self).__init__()
        self.initialLayers = initialLayers
        self.finalLayer = nn.Linear(1280   , outputClasses)
        self.softmax = nn.Softmax(1)
        self.pool = nn.AvgPool2d(7, stride = 1)
        
    def forward(self, x):
        x = self.initialLayers(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.finalLayer(x)
        x = self.softmax(x)
        return x
    

In [5]:
model = newMobilenet(5, initialLayers)
model = model.to(cuda)

#recommended transforms to be used by pytorch
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
error = nn.CrossEntropyLoss()

In [6]:
#Trains the model with a particular optimizer, error function and image transform
def train(model, optimizer, error, TRANSFORM_IMG):
    EPOCHS = 10
    model.train()
    train = datasets.ImageFolder("train", transform =TRANSFORM_IMG )
    batchSize = 32
    train = DataLoader(train, batch_size=batchSize, shuffle = True)
    for epoch in range(EPOCHS):
        correct = 0
        epochLoss = 0
        for idx, (xBatch, yBatch) in enumerate(train):
            xBatch = xBatch.view(-1, 3, 224, 224).to(cuda)
            yBatch = yBatch.to(cuda)
            output = model(xBatch)
            loss = error(output, yBatch)
            epochLoss += loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(f'Epoch {epoch} has loss {epochLoss}')

In [7]:
#Tests a models performance on a test set
def test(model, TRANSFORM_IMG):
    test = datasets.ImageFolder("test", transform =TRANSFORM_IMG )
    test = DataLoader(test, batch_size=1)
    actual = {}
    predicted = {}
    classes = [i for i in range(5)]
    for c in classes:
        actual[c] = 0
        predicted[c] = 0
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for x, y in test:
            total += 1
            output = model(x.to(cuda))
            chosen = torch.argmax(output)
            actual[y.item()] += 1
            predicted[chosen.item()] += 1
            if chosen == y.to(cuda):
                correct += 1
        print(f'Accuracy: {correct / total}%')
    print(actual)
    print(predicted)

In [8]:
train(model, optimizer, error, preprocess)


Epoch 0 has loss 289.0041806101799
Epoch 1 has loss 285.2759078145027
Epoch 2 has loss 282.3076863884926
Epoch 3 has loss 281.24393421411514
Epoch 4 has loss 282.48833084106445
Epoch 5 has loss 282.70259684324265
Epoch 6 has loss 282.1553084850311
Epoch 7 has loss 280.85941302776337
Epoch 8 has loss 287.19952768087387
Epoch 9 has loss 281.5999028682709


KeyboardInterrupt: 

In [9]:
test(model, preprocess)

Accuracy: 0.9393518518518519%
{0: 522, 1: 528, 2: 258, 3: 438, 4: 414}
{0: 582, 1: 554, 2: 276, 3: 336, 4: 412}


In [10]:
#saves the model for later inference.
torch.save(model.state_dict(), 'transferLearnedModel.pth')